# 1 - Prepare the knowledge base
This step is to find the relevant content to the question and prepare the knowledge base(context) for GenAI for reference. To index documents with a search engine. 

In [1]:
import minsearch

- TO understand how to read doc via a search engine built before
https://github.com/alexeygrigorev/minsearch

In [2]:
import json

In [3]:
with open('documents.json','rt') as f_in: # read json file in reading text mode as file input
    docs_raw = json.load(f_in)

In [178]:
# docs_raw

In [5]:
len(docs_raw) # means it has 3 different courses 

3

In [6]:
documents = []

for course_dict in docs_raw: 
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc) 

#### 3 steps for this loop:
# 1. Store the flattened course_dict['documents'] in each doc as a dictionary,
# 2. Add a new key'course' in each doc. 
# 3. Append doc to documents dict. 


In [7]:
documents[0:2]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'}]

In [8]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)
# filter is "course" here 

In [44]:
q = 'the course has already started, can I still be able to enroll now ? '

In [45]:
index.fit(documents)
# fit() function is from scikit-learn, here is to parse the schema 

In [46]:
#  boost : if some content is more important in the text_fields, apply different weight on it.
#  normally give important one 1.0 ~ 3.0 
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    filter_dict = {'course': 'data-engineering-zoomcamp'}, # optional
    boost_dict = boost, 
    num_results = 5
)

In [47]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# 2 - Connect to GenAI API

## prepare the prompt

In [48]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION:{question}

CONTEXT:
{context}
""".strip()
# .strip() to remove leading and trailing whitespace characters from a string — including spaces, tabs (\t), and newline characters (\n).

In [49]:
context = ""

for doc in results: 
    context += f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"

In [50]:
print(context)

section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question:Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question:Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the 

In [51]:
prompt = prompt_template.format(question=q, context=context).strip()

In [52]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION:the course has already started, can I still be able to enroll now ? 

CONTEXT:
section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question:Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start workin

## 1) Gemini AI API
- in terminal : ```pip install -q -U google-genai```
- set environment variable, export GEMINI_API_KEY= "..."

In [53]:
from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        types.Content(
            role="user",
            parts=[types.Part(text= prompt)])],
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0)), # Disables thinking to minimize the cost, otherwise remove this config 
)

print(response.text)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


## 2) Groq AI API
- set environment variable, export GROQ_API_KEY="..."

In [54]:
from groq import Groq
import os

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

response = client.chat.completions.create(
    messages=[{"role":"user", "content": prompt}], # keyword is "messages"
    model="llama-3.3-70b-versatile",
    stream=False,
)

print(response.choices[0].message.content)

Yes, even if you don't register, you're still eligible to submit the homeworks.


# 3 - Wrap up and modularze the code

In [111]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'}, # optional
        boost_dict = boost, 
        num_results = 10
    )
    return results

In [112]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION:{question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results: 
        context += f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [171]:
def llm_gemini(prompt):
    client_gemini = genai.Client()
    response = client_gemini.models.generate_content(
        model="gemini-2.5-flash",
        contents=[
            types.Content(
                role="user",
                parts=[types.Part(text= prompt)])],
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=0)), # Disables thinking to minimize the cost, otherwise remove this config 
        )

    return response.text

In [172]:
def llm_groq(prompt):
    client_groq = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )
    
    response = client_groq.chat.completions.create(
        messages=[{"role":"user", "content": prompt}], # keyword is "messages"
        model="llama-3.3-70b-versatile",
        stream=False,
    )
    
    return response.choices[0].message.content

In [173]:
query = "how to run python with kafka?"
# query = "Can I still enroll?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer_gemini = llm_gemini(prompt)
    answer_groq = llm_groq(prompt)
    return (
        "========== GEMINI ANSWER ==========\n"
        f"{answer_gemini}\n\n"
        "========== GROQ ANSWER ==========\n"
        f"{answer_groq}"
    )

In [174]:
rag(query)

'========== GEMINI ANSWER ==========\nTo run Python with Kafka, you need to install the necessary dependencies:\n\n*   `confluent-kafka`: Install with `pip install confluent-kafka` or `conda install conda-forge::python-confluent-kafka`.\n*   `fastavro`: Install with `pip install fastavro`.\n\nIf you encounter a "Module kafka not found" error, create a virtual environment, activate it, and install dependencies from `requirements.txt`:\n```bash\npython -m venv env\nsource env/bin/activate  # Use env/Scripts/activate for Windows\npip install -r ../requirements.txt\n```\nIf you face an error "ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'", it is suggested to use `pip install kafka-python-ng` instead of `kafka-python`.\n\n========== GROQ ANSWER ==========\nTo run Python with Kafka, you need to install the necessary dependencies. \n\nYou can install `confluent-kafka` using `pip install confluent-kafka` or `conda install conda-forge::python-confluent-kafka`. \n\nAdditionally

In [176]:
print(rag(query))

========== GEMINI ANSWER ==========
To run Python with Kafka, you need to install the necessary dependencies:

*   For `confluent-kafka`: `pip install confluent-kafka` or `conda install conda-forge::python-confluent-kafka`
*   For `fastavro`: `pip install fastavro`

If you encounter a "Module 'kafka' not found" error, create and activate a virtual environment, then install the packages from `requirements.txt`.

Steps for virtual environment:
1.  Create: `python -m venv env`
2.  Activate (MacOS/Linux): `source env/bin/activate`
3.  Activate (Windows): `env/Scripts/activate`
4.  Install dependencies: `pip install -r ../requirements.txt`
5.  Deactivate: `deactivate`

Note that the Faust library is no longer maintained. Alternatively, you can follow Python videos 6.13 & 6.14 or use the RedPanda Python version.

========== GROQ ANSWER ==========
To run Python with Kafka, you need to install the required dependencies. You can install `confluent-kafka` using `pip install confluent-kafka` or `

In [177]:
print(rag("the class already started, can I still enroll? "))

========== GEMINI ANSWER ==========
Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

========== GROQ ANSWER ==========
Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


In [144]:
answer_groq = llm_groq(prompt)

To run Python with Kafka, you need to:

1. Install the necessary dependencies, which are:
   - `confluent-kafka`: Install using `pip install confluent-kafka` or `conda install conda-forge::python-confluent-kafka`
   - `fastavro`: Install using `pip install fastavro`

2. Create a virtual environment and install the requirements from `requirements.txt` to avoid dependency conflicts.

3. Activate the virtual environment before running your Python scripts. 

Note: Docker images should be up and running before creating the virtual environment. 

For more specific instructions on running Python files, ensure that you are in the correct directory and that the Kafka environment is properly set up. If you encounter issues like "ModuleNotFoundError: No module named 'kafka.vendor.six.moves'", consider using `kafka-python-ng` instead.
